# Test CLI

````
python ../q2_ritme/split_train_test.py data_splits data/metadata_proc_v20240323_r0_r3_le_2yrs.tsv data/all_otu_table_filt.qza host_id --seed 12
````

````
python ../q2_ritme/find_best_model_config.py ../config/r_local_linreg.json data_splits/train_val.pkl --path-to-tax data/otu_taxonomy_all.qza --path-to-tree-phylo data/silva-138-99-rooted-tree.qza --path-store-model-logs ritme_refact_logs
````

# Test Python API

In [3]:
from q2_ritme.find_best_model_config import (
    _load_experiment_config,
    _load_phylogeny,
    _load_taxonomy,
    find_best_model_config,
)
from q2_ritme.split_train_test import _load_data, split_train_test

%load_ext autoreload

In [4]:
model_config_path = "../config/r_local_linreg.json"
config = _load_experiment_config(model_config_path)

## Read data

In [5]:
md, ft = _load_data(config["path_to_md"], config["path_to_ft"])
print(md.shape, ft.shape)

(4151, 74) (9478, 5580)


## Split

In [6]:
train_val, test = split_train_test(
    md,
    ft,
    stratify_by_column=config["stratify_by_column"],
    feature_prefix=config["feature_prefix"],
    train_size=config["train_size"],
    seed=config["seed_data"],
)

/Users/adamova/Documents/projects/14_LM1/q2-ritme/q2_ritme/split_train_test.py:135: UserWarning: Provided feature table contains absolute instead of relative abundances. Hence, converting it to relative abundances...
  warnings.warn(


Train: (3170, 5654), Test: (779, 5654)


## Find best model config

In [7]:
tax = _load_taxonomy(config["path_to_tax"])
phylo = _load_phylogeny(config["path_to_phylo"])

best_model_dict, path_to_exp = find_best_model_config(
    config, train_val, tax, phylo, path_store_model_logs="ritme_refact_logs"
)

(train_linreg pid=40081) (3170, 75)
(train_linreg pid=40081) Train: (2566, 75), Test: (604, 75)


2024-10-18 18:29:07,308	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/adamova/Documents/projects/14_LM1/q2-ritme/experiments/ritme_refact_logs/r_local_linreg_mlruns/linreg' in 0.0157s.


(train_linreg pid=40100) (3170, 129)
(train_linreg pid=40100) Train: (2566, 129), Test: (604, 129)


2024-10-18 18:29:07,315	INFO tune.py:1041 -- Total run time: 29.34 seconds (29.24 seconds for the tuning loop).


In [8]:
path_to_exp

'ritme_refact_logs/r_local_linreg_mlruns'

In [9]:
best_model_dict

{'linreg': <q2_ritme.evaluate_models.TunedModel at 0x19de710a0>}